In [ ]:
import pickle
import pandas as pd
from tasks import wsd
from utils import nlp_tools
from tqdm.auto import tqdm
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

tqdm.pandas()

with open("data/machine_nn01_all.pickle", "rb") as f:
    df = pickle.load(f)

In [ ]:
definition_df = df[['lemma','sense_id','definition']]
definition_df = definition_df[definition_df.lemma=="machine"]
definition_df.drop_duplicates(inplace = True)
definition_df = definition_df.reset_index(drop=True)
definition_df["nlp_definition"] = definition_df.apply (lambda row: nlp_tools.preprocess(row["definition"]), axis=1)

print (len(definition_df))

In [ ]:
definition_df.head()

In [ ]:
machine_df = df[df.lemma=="machine"]
machine_df = machine_df.reset_index(drop=True)
machine_df["nlp_full_text"] = machine_df.apply (lambda row: nlp_tools.preprocess(row["text"]["full_text"]), axis=1)
machine_df.head()

In [ ]:
approach = "random_predict"

machine_df[approach] = machine_df.progress_apply (lambda row: wsd.random_predict(definition_df), axis=1)

wsd.eval(machine_df[approach],machine_df["sense_id"])

In [ ]:
# be careful: i am using the example sentence (row["text"]["full_text"]) as the input sentence and then measure its word overlap with the definition (see function). if you instead want to use the example as training data, we need to split in train/test

approach = "def_tok_overlap_ranking"

machine_df[approach] = machine_df.progress_apply (lambda row: wsd.tok_overlap_ranking(row["nlp_full_text"], definition_df), axis=1)

wsd.eval(machine_df[approach],machine_df["sense_id"])

In [ ]:
approach = "sent_embedding"

machine_df[approach] = machine_df.progress_apply (lambda row: wsd.sent_embedding(row["nlp_full_text"], definition_df), axis=1)

wsd.eval(machine_df[approach],machine_df["sense_id"])

In [ ]:
approach = "w2v_lesk_ranking"

# Warning: I use a Word2vec model trained on all 19thC BL corpus that is locally stored.
wemb_model = Word2Vec.load("models/w2v/w2v_v004/w2v_words.model")
machine_df[approach] = machine_df.progress_apply (lambda row: wsd.w2v_lesk_ranking(row["nlp_full_text"], definition_df, wemb_model), axis=1)

wsd.eval(machine_df[approach],machine_df["sense_id"])

In [ ]:
approach = "bert_lesk_ranking"

# Download model from (warning: this is a contemporary model):
# https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/bert-base-nli-mean-tokens.zip
bert_sentsim_model = SentenceTransformer('models/bert/bert-base-nli-mean-tokens')
machine_df[approach] = machine_df.progress_apply (lambda row: wsd.bert_lesk_ranking(row["text"]["full_text"], definition_df, bert_sentsim_model), axis=1)

wsd.eval(machine_df[approach],machine_df["sense_id"])